In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
)

import jsonref

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")


In [2]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

In [3]:
# Initlizing the OpenaAPI Tool

with open("./weather_openapi.json", "r") as f:
        openapi_weather = jsonref.loads(f.read())

# Initialize agent OpenApi tool using the read in OpenAPI spec
weather_tool = {
        "type": "openapi",
        "openapi":{
            "name": "weather",
            "spec": openapi_weather,
            "auth": {
                "type": "anonymous"
            },
        }
}

In [4]:
# setting up the agent with opeanAPI tool specfication

agent_name = "weather-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful Weather Agent that provides weather information using the provided OpenAPI Tool",
        tools=[weather_tool]
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

Agent created (id: weather-agent:1, name: weather-agent, version: 1)


In [5]:
# Creating a conversation object

# creating an openai client first
openai_client = project_client.get_openai_client()

# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

Created conversation with id: conv_b48faa914db8123000hsQ6Vs31z23uMhW7MBXbs3JsmKuCT8le


In [6]:
user_input = "What's the weather like in New York City today?"

In [7]:
response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )
print(f"Weather Agent: {response.output_text}")

Weather Agent: The weather in New York City today is clear with a temperature of around 28°F (-2°C). The wind is coming from the west-southwest at 15 mph (23 km/h), making it feel like 17°F (-9°C). The humidity is 40%, and the visibility is good at 9 miles (16 km). There's no precipitation expected, and the UV index is 0.
